## Imports de Bibliotecas

In [1]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os
import nrrd
from matplotlib.backends.backend_pdf import PdfPages

## Funções que geram os recortes de cada fatia

#### Só o T1

In [2]:
# corta imagem 2d, de modo a eliminar pixels pretos, deixando só um retângulo ajustado ao cérebro
def crop_black_slices_2d(data):
        if np.count_nonzero(data) == 0:
                return [], []
        
        right, left, top, down = data.shape[1]-1, 0, 0, data.shape[0]-1

        while np.count_nonzero(data[:, left]) == 0: # anda com limite esquerdo pra direita até achar um pixel não preto
                left += 1

        while np.count_nonzero(data[:, right]) == 0: # anda com limite direito pra esquerda até achar um pixel não preto
                right -= 1

        while np.count_nonzero(data[top, :]) == 0: # anda com limite inferior pra cima até achar um pixel não preto
                down += 1

        while np.count_nonzero(data[down, :]) == 0: # anda com limite superior pra baixo até achar um pixel não preto
                top -= 1

        data_2d = data[down:top, left:right]

        return data_2d, [left, right, top, down]

# cria o grid, encaixando os quadrados até o eixo central
def create_grid_half_vertical(data, size, overlap, threshold):
    grid = []
    width = data.shape[1]//2
    
    for i in range(0, (data.shape[0] - size + 1), size):
        for j in range(width, 0, -overlap):
            if j - size < 0:  # Garante que o quadrado não atravesse o centro
                break
            if np.count_nonzero(data[i:i + size - 1, j - size - 1:j]) > data[i:i + size - 1, j - size - 1:j].size * threshold:
                grid.append([i, i + size - 1, j - size - 1, j])  # [y1, y2, x1, x2]
    
    return grid

# cria o grid, encaixando os quadrados até o eixo central
def create_grid_half_vertical_flexible(data, size, overlap, threshold):
    grid = []
    width = data.shape[1]//2
    
    for i in range(0, (data.shape[0] - size + 1), size):
        for j in range(width, 0, -overlap):
            if j - size < 0:  # Garante que o quadrado não atravesse o centro
                break
            if np.count_nonzero(data[i:i + size - 1, j - size - 1:j]) > data[i:i + size - 1, j - size - 1:j].size * threshold:
                grid.append([i, i + size - 1, j - size - 1, j])  # [y1, y2, x1, x2]
    
    return grid

# alinha o grid das pontas até encontrar o cérebro, evitando recortes puramente pretos
# implementar: descarte de fatias com poucos pixels, tratamento de casos onde um quadradinho nunca acha cérebro, então anda até passar do limite do vetor e achar erro
def move_grid(data, grid):
    new_grid = []
    width = data.shape[1]//2

    for rect in grid:
        y1, y2, x1, x2 = rect[0], rect[1], rect[2], rect[3]
        
        # Verifica se o quadrado contém apenas pixels pretos
        if np.count_nonzero(data[y1:y2+1, x1:x2+1]) < 0:
            break

        # Movimenta o grid para evitar regiões completamente pretas
        while y2 < data.shape[0] - 1:
            if np.count_nonzero(data[y1, x1:x2+1]) > 0:  # Se há conteúdo, sai do loop
                break
            y1 += 1
            y2 += 1

        while y1 > 0:
            if np.count_nonzero(data[y2, x1:x2+1]) > 0:  # Se há conteúdo, sai do loop
                break
            y1 -= 1
            y2 -= 1

        while x1 > 0 and  x2 < width - 1:
            if np.count_nonzero(data[y1:y2+1, x1]) > 0:  # Se há conteúdo, sai do loop
                break
            x1 += 1
            x2 += 1

        while x1 > 0 and x2 < width:
            if np.count_nonzero(data[y1:y2+1, x2]) > 0:  # Se há conteúdo, sai do loop
                break
            x1 -= 1
            x2 -= 1

        # Após movimentar, verifica se o quadrado contém pixels diferentes de preto
        if (
            0 <= y1 < data.shape[0]
            and 0 <= y2 < data.shape[0]
            and 0 <= x1 < data.shape[1]
            and 0 <= x2 < data.shape[1]
            and np.count_nonzero(data[y1:y2+1, x1:x2+1]) >= data[y1:y2+1, x1:x2+1].size * 0.4
        ):
            new_grid.append([y1, y2, x1, x2])

    return new_grid


# espeçha o grid
# a ideia é a funcao create_grid_half criar metade do grid, e a outra metade ser espelhada com essa função
def mirror_grid_vertical(grid, data):
    shape = data.shape  # Obtém o formato da imagem (altura, largura)
    mirror = []  # Lista para armazenar o grid espelhado

    for rect in grid:
        # Extrai as coordenadas do retângulo original
        y1, y2, x1, x2 = rect

        # Calcula as novas coordenadas espelhadas em relação ao eixo vertical (colunas)
        x3 = shape[1] - x2  # Posição inicial do espelhado
        x4 = shape[1] - x1  # Posição final do espelhado
        y3, y4 = y1, y2  # Linhas permanecem iguais

        # Garante que as novas coordenadas estão na ordem correta
        if x3 > x4:
            x3, x4 = x4, x3

        # Adiciona o retângulo espelhado
        mirror.append([y3, y4, x3, x4])

    return mirror

def create_full_grid(data, size, overlap, threshold):
        moved_grid, mirror_grid = [], []
        
        if np.count_nonzero(data) != 0 :
        
            grid_2d = create_grid_half_vertical(data, size, overlap, threshold)
            moved_grid = move_grid(data, grid_2d)
            mirror_grid = mirror_grid_vertical(moved_grid, data)

        return moved_grid, mirror_grid

# pseudocodigo
# cria grid completo
#   criar correspondência de índices para pegar esquerdo e direito  
# pra cada quadrado testa lado pra saber se é esquerda ou direito
#   se esquerdo, mexe pra direita e vice-versa
#   daí move os dois juntos até um encontrar um limite(parece o melhor, permitindo uns tiquinhos mínimos pretos) 
#   ou cada um (asimétrico, mas encaixaria perfeitmaente)

# cria o grid, encaixando os quadrados até o eixo central
def create_left_right_grid(data, size, overlap, threshold):
    grid_l = []
    grid_r = []
    width = data.shape[1]//2
    height = data.shape[0]//2
    
    for i in range(height - size - 1, 0, -overlap):
        for j in range(width, size, -overlap):
            square_left = data[i+1:i+size, j - size + 1:j]
            square_right = data[i+1:i+size, width + (width - j) + 1:width + (width - j) + size]

            if j - size < 0:  # Garante que o quadrado não atravesse o centro
                break

            if np.count_nonzero(square_left) > square_left.size * threshold or np.count_nonzero(square_right) > square_left.size * threshold:
                grid_l.append([i+1, i + size, j - size + 1, j])  # [y1, y2, x1, x2]
                grid_r.append([i+1, i + size, width + 1 + (width - j), width + (width - j) + size])

    for i in range(height, height*2, overlap):
        for j in range(width, 0, -overlap):
            square_left = data[i:i+size-1, j - size + 1:j]
            square_right = data[i:i+size-1, width + (width - j) + 1:width + (width - j) + size]

            if j - size < 0:  # Garante que o quadrado não atravesse o centro
                break

            if np.count_nonzero(square_left) > square_left.size * threshold or np.count_nonzero(square_right) > square_left.size * threshold:
                grid_l.append([i, i + size-1, j - size + 1, j])  # [y1, y2, x1, x2]
                grid_r.append([i, i + size-1, width + (width - j) + 1, width + (width - j) + size])

    return grid_l, grid_r

def is_border_empty(data, x1, x2, y1, y2):
    """Verifica se as bordas do retângulo estão vazias."""
    return (np.count_nonzero(data[y1, x1:x2]) == 0 and
            np.count_nonzero(data[y2, x1:x2]) == 0 and
            np.count_nonzero(data[y1:y2, x1]) == 0 and
            np.count_nonzero(data[y1:y2, x2]) == 0)

def is_region_dense(data, x1, x2, y1, y2, thresh):
    """Verifica se a região interna do retângulo possui densidade mínima."""
    region = data[y1:y2, x1:x2]
    return np.count_nonzero(region) >= region.size * thresh

#i e n//2 + i
def move_grid_full(data, grid_l, grid_r, tresh_move=0.4):
    new_grid_l = []
    new_grid_r = []
    width = data.shape[1]//2
    n = len(grid_l)

    #print(n)

    for idx in range(0, n):
        rect_left = grid_l[idx]
        rect_right = grid_r[idx]
        #print(f"{idx} e {n//2+idx}")

        y1, y2, x1, x2 = rect_left[0], rect_left[1], rect_left[2], rect_left[3]
        x3, x4 = rect_right[2], rect_right[3]
        
        # Verifica se o quadrado contém apenas pixels pretos
        if np.count_nonzero(data[y1:y2, x1:x2]) <= 0 and np.count_nonzero(data[y1:y2, x3:x4]) <= 0:
            break

        if ((is_border_empty(data, x1, x2, y1, y2) and  is_region_dense(data, x1, x2, y1, y2, tresh_move)) 
            or (is_border_empty(data, x3, x4, y1, y2) and is_region_dense(data, x3, x4, y1, y2, tresh_move))):
            break

        # se estiver pra cima, vai descer
        while y2 < data.shape[0] - 1:
            if np.count_nonzero(data[y1, x1:x2]) > 0 or np.count_nonzero(data[y1, x3:x4]) > 0:  # Se há conteúdo, sai do loop
                break
            y1 += 1
            y2 += 1

        # se estiver pra baixo, vai subir
        while y1 > 0:
            if np.count_nonzero(data[y2, x1:x2]) > 0 or np.count_nonzero(data[y2, x3:x4]) > 0:  # Se há conteúdo, sai do loop
                break
            y1 -= 1
            y2 -= 1

        while x2 < width and  x3 > width + 1:
            if np.count_nonzero(data[y1:y2, x1]) > 0 or np.count_nonzero(data[y1:y2, x4]) > 0:  # Se há conteúdo, sai do loop
                break
            x1 += 1
            x2 += 1

            x3 -= 1
            x4 -= 1

        # Após movimentar, verifica se o quadrado contém pixels diferentes de preto
        if (
            0 <= y1 < data.shape[0]
            and 0 <= y2 < data.shape[0]
            and 0 <= x1 < data.shape[1]
            and 0 <= x2 < data.shape[1]
            and (np.count_nonzero(data[y1:y2, x1:x2]) >= data[y1:y2, x1:x2].size * tresh_move
            or np.count_nonzero(data[y1:y2, x3:x4]) >= data[y1:y2, x3:x4].size * tresh_move)
        ):
            new_grid_l.append([y1, y2, x1, x2])
            new_grid_r.append([y1, y2, x3, x4])

    return new_grid_l, new_grid_r

def create_full_grid_no_mirror(data, size, overlap, threshold, thresh_move=0.4):
    grid_l, grid_r = create_left_right_grid(data, size, overlap, threshold)

    grid_l, grid_r = move_grid_full(data, grid_l, grid_r, thresh_move)
    
    return grid_l, grid_r

def plot_grid(data, grid, idx):
    if (data.size != 0 and len(grid) != 0):
        plt.imshow(data, cmap='gray')
        plt.title(f"slice {idx}")

        for rect in (grid):
        
            y1, y2, x1, x2 = rect[0], rect[1], rect[2], rect[3]

            plt.gca().add_patch(plt.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=1, edgecolor='red', facecolor='none'))

        plt.show()

#### Todas as 3 modalidades: T1, Flair, T2

In [5]:
# corta imagem 2d, de modo a eliminar pixels pretos, deixando só um retângulo ajustado ao cérebro
def crop_black_slices_2d(data):
    if np.count_nonzero(data) == 0:
        return data, [] # Retorna o dado original se for tudo zero para evitar erro no slicing depois
    
    rows = np.any(data, axis=1)
    cols = np.any(data, axis=0)
    if not np.any(rows) or not np.any(cols): # Se não houver pixels não pretos
        return data, []

    ymin, ymax = np.where(rows)[0][[0, -1]]
    xmin, xmax = np.where(cols)[0][[0, -1]]
    
    # Adiciona +1 a ymax e xmax porque o slicing é exclusivo no limite superior
    data_2d = data[ymin:ymax+1, xmin:xmax+1]
    return data_2d, [xmin, xmax, ymin, ymax]

def create_left_right_grid(data, size, overlap, threshold):
    grid_l = []
    grid_r = []
    
    if data.ndim != 2 or data.shape[0] < size or data.shape[1] < size :
        return grid_l, grid_r

    height, width_full = data.shape
    width_half = width_full // 2

    # Esta é a segunda lógica de loop que você tinha, que parece mais correta para gerar pares.
    for i in range(0, height - size + 1, overlap): # Itera nas linhas
        for j_offset in range(0, width_half - size + 1, overlap): # Itera nas colunas para o lado esquerdo
            x1_l = width_half - size - j_offset
            x2_l = width_half - j_offset 
            if x1_l < 0: continue # Garante que x1_l não seja negativo

            square_left = data[i : i + size, x1_l : x2_l] # x2_l é exclusivo no slicing
            if square_left.size == 0: continue
            
            x1_r = width_half + j_offset
            x2_r = width_half + size + j_offset
            if x2_r > width_full: continue # Garante que x2_r não exceda a largura

            square_right = data[i : i + size, x1_r : x2_r] # x2_r é exclusivo
            if square_right.size == 0: continue
            
            if np.count_nonzero(square_left) > square_left.size * threshold or \
               np.count_nonzero(square_right) > square_right.size * threshold:
                # Coordenadas salvas são inclusivas para y2 e x2
                grid_l.append([i, i + size - 1, x1_l, x2_l - 1]) 
                grid_r.append([i, i + size - 1, x1_r, x2_r - 1])
                
    return grid_l, grid_r

def is_border_empty(data, x1, x2, y1, y2): # x2, y2 são inclusivos aqui
    if y1 < 0 or y2 >= data.shape[0] or x1 < 0 or x2 >= data.shape[1]: return True
    if y1 > y2 or x1 > x2 : return True 

    empty = True
    # Para count_nonzero em uma linha/coluna, o slice final precisa ser +1
    if y2 - y1 >= 0 and x2 - x1 >= 0: 
        empty &= (np.count_nonzero(data[y1, x1:x2+1]) == 0) 
        empty &= (np.count_nonzero(data[y2, x1:x2+1]) == 0) 
        empty &= (np.count_nonzero(data[y1:y2+1, x1]) == 0) 
        empty &= (np.count_nonzero(data[y1:y2+1, x2]) == 0) 
    return empty

def is_region_dense(data, x1, x2, y1, y2, thresh): # x2, y2 são inclusivos
    if y1 < 0 or y2 >= data.shape[0] or x1 < 0 or x2 >= data.shape[1]: return False
    if y1 > y2 or x1 > x2 : return False
    region = data[y1:y2+1, x1:x2+1] 
    if region.size == 0: return False
    return np.count_nonzero(region) >= region.size * thresh

def move_grid_full(data, grid_l, grid_r, tresh_move=0.4):
    new_grid_l = []
    new_grid_r = []
    
    if not grid_l or not grid_r or len(grid_l) != len(grid_r):
        return new_grid_l, new_grid_r

    height_img, width_img = data.shape
    width_half = width_img // 2

    for idx in range(len(grid_l)):
        rect_left = grid_l[idx]
        rect_right = grid_r[idx]

        y1_orig, y2_orig, x1_l_orig, x2_l_orig = rect_left 
        _, _, x1_r_orig, x2_r_orig = rect_right  

        y1, y2, x1_l, x2_l = y1_orig, y2_orig, x1_l_orig, x2_l_orig
        x1_r, x2_r = x1_r_orig, x2_r_orig
        
        patch_l_orig = data[y1_orig : y2_orig + 1, x1_l_orig : x2_l_orig + 1]
        patch_r_orig = data[y1_orig : y2_orig + 1, x1_r_orig : x2_r_orig + 1]

        # A condição original `np.count_nonzero(data[y1:y2, x1:x2]) <= 0` tinha um bug de slicing (y2, x2 exclusivo)
        # e era `data[y1:y2, x1:x2]` em vez de patches individuais.
        if patch_l_orig.size > 0 and patch_r_orig.size > 0: # Evita erro se o patch for 0xN ou Nx0
            if np.count_nonzero(patch_l_orig) == 0 and np.count_nonzero(patch_r_orig) == 0: # Modificado para == 0
               continue
        elif patch_l_orig.size == 0 and patch_r_orig.size == 0: # Se ambos os patches não têm tamanho
            continue


        # Sua condição de parada com is_border_empty e is_region_dense:
        # O if original era: `if ((is_border_empty(data, x1, x2, y1, y2) and  is_region_dense(data, x1, x2, y1, y2, tresh_move)) 
        #                       or (is_border_empty(data, x3, x4, y1, y2) and is_region_dense(data, x3, x4, y1, y2, tresh_move))): break`
        # x2,x4 são os limites direitos dos patches esquerdo e direito.
        # Acredito que a intenção é parar se um patch já está bem posicionado (borda vazia, mas interior denso)
        # Aplicando a x1_l, x2_l e x1_r, x2_r
        if (is_border_empty(data, x1_l, x2_l, y1, y2) and is_region_dense(data, x1_l, x2_l, y1, y2, tresh_move)) or \
           (is_border_empty(data, x1_r, x2_r, y1, y2) and is_region_dense(data, x1_r, x2_r, y1, y2, tresh_move)):
            # Se um dos patches já satisfaz, não movemos e adicionamos se for denso no final?
            # Ou só adicionamos este par e continuamos? A lógica original tinha `break` que sairia do loop de `idx`.
            # Vou remover o `break` e deixar a validação final decidir.
            pass


        # Movimento Vertical (y1, y2 são compartilhados)
        y1_temp_v, y2_temp_v = y1, y2
        while y2_temp_v < height_img - 1: # Mover para baixo
            if np.count_nonzero(data[y1_temp_v, x1_l:x2_l+1]) > 0 or np.count_nonzero(data[y1_temp_v, x1_r:x2_r+1]) > 0:
                break
            y1_temp_v += 1
            y2_temp_v += 1
        y1, y2 = y1_temp_v, y2_temp_v

        y1_temp_v, y2_temp_v = y1, y2 # Reset para a nova posição y
        while y1_temp_v > 0: # Mover para cima
            if np.count_nonzero(data[y2_temp_v, x1_l:x2_l+1]) > 0 or np.count_nonzero(data[y2_temp_v, x1_r:x2_r+1]) > 0:
                break
            y1_temp_v -= 1
            y2_temp_v -= 1
        y1, y2 = y1_temp_v, y2_temp_v

        # Movimento Horizontal (x1_l, x2_l para patch esquerdo; x1_r, x2_r para patch direito)
        # Tentando mover os patches para o CENTRO se suas bordas EXTERNAS estiverem vazias.
        x1_l_temp_h, x2_l_temp_h = x1_l, x2_l
        x1_r_temp_h, x2_r_temp_h = x1_r, x2_r

        # Mover patch esquerdo para a direita (em direção ao centro)
        while x2_l_temp_h < width_half -1 : # Não deixar cruzar/tocar o centro
            if np.count_nonzero(data[y1:y2+1, x1_l_temp_h]) > 0: # Se a borda mais à esquerda (x1_l) tem conteúdo
                break
            x1_l_temp_h +=1
            x2_l_temp_h +=1
        
        # Mover patch direito para a esquerda (em direção ao centro)
        while x1_r_temp_h > width_half : # Não deixar cruzar/tocar o centro (width_half é o primeiro pixel da direita se N par)
            if np.count_nonzero(data[y1:y2+1, x2_r_temp_h]) > 0: # Se a borda mais à direita (x2_r) tem conteúdo
                break
            x1_r_temp_h -=1
            x2_r_temp_h -=1
        
        x1_l, x2_l = x1_l_temp_h, x2_l_temp_h
        x1_r, x2_r = x1_r_temp_h, x2_r_temp_h


        # Validação final após todos os movimentos
        # Coordenadas do grid são [y_inicio, y_fim_inclusivo, x_inicio, x_fim_inclusivo]
        # Valid_l e Valid_r usam estas coordenadas inclusivas para is_region_dense
        valid_l = is_region_dense(data, x1_l, x2_l, y1, y2, tresh_move)
        valid_r = is_region_dense(data, x1_r, x2_r, y1, y2, tresh_move)

        if valid_l or valid_r: 
            if valid_l: # Se o esquerdo é válido, adicione-o
                new_grid_l.append([y1, y2, x1_l, x2_l])
            else: # Se o esquerdo não for válido, mas o direito sim
                  # Adicionar o rect_left original para manter o par, mesmo que não seja denso.
                  # Ou você pode decidir não adicionar se não for válido.
                  # Para consistência de pares, vamos adicionar o original se o seu par for válido.
                new_grid_l.append(rect_left) 

            if valid_r: # Se o direito é válido, adicione-o
                new_grid_r.append([y1, y2, x1_r, x2_r])
            else: # Se o direito não for válido, mas o esquerdo sim
                new_grid_r.append(rect_right)
                
    return new_grid_l, new_grid_r

def create_full_grid_no_mirror(data, size, overlap, threshold, thresh_move=0.4):
    # Esta é a definição que você forneceu que chama as outras funções.
    grid_l, grid_r = create_left_right_grid(data, size, overlap, threshold)
    # É importante que move_grid_full lide com grids vazios ou de tamanhos diferentes se isso puder ocorrer.
    # A implementação de move_grid_full acima já tem uma verificação para isso.
    moved_grid_l, moved_grid_r = move_grid_full(data, grid_l, grid_r, thresh_move)
    return moved_grid_l, moved_grid_r

def create_grid_half_vertical(data, size, overlap, threshold):
    grid = []
    if data.ndim != 2 or data.shape[0] < size or data.shape[1] < size//2 : # Checagem básica
        return grid
    width = data.shape[1]//2
    
    for i in range(0, (data.shape[0] - size + 1), size): # O step aqui é 'size', não 'overlap'
        for j in range(width, 0, -overlap): # j é o limite DIREITO do patch potencial
            x1 = j - size
            x2 = j 
            if x1 < 0: # Garante que o quadrado não atravesse o limite esquerdo
                # Se x1 é negativo, o último patch válido teria x1=0.
                # Poderia ajustar j para o último válido, ou simplesmente 'break'/'continue'
                break # Sai do loop interno de 'j' se não couber mais
            
            # Slicing é data[y1:y2_exclusivo, x1:x2_exclusivo]
            # Coords salvas são [y1_inclusivo, y2_inclusivo, x1_inclusivo, x2_inclusivo]
            patch_data = data[i : i + size, x1 : x2]
            if patch_data.size == 0: continue

            if np.count_nonzero(patch_data) > patch_data.size * threshold:
                grid.append([i, i + size - 1, x1, x2 - 1]) 
    return grid

def create_grid_half_vertical_flexible(data, size, overlap, threshold):
    grid = []
    if data.ndim != 2 or data.shape[0] < size or data.shape[1] < size//2 :
        return grid
    width = data.shape[1]//2
    
    for i in range(0, (data.shape[0] - size + 1), overlap): # overlap no passo de 'i'
        for j in range(width, 0, -overlap): # j é o limite DIREITO
            x1 = j - size
            x2 = j
            if x1 < 0:
                break
            
            patch_data = data[i : i + size, x1 : x2]
            if patch_data.size == 0: continue

            if np.count_nonzero(patch_data) > patch_data.size * threshold:
                grid.append([i, i + size - 1, x1, x2 - 1])
    return grid

def plot_grid(data, grid_l, grid_r, slice_idx_display): # Modificado para aceitar grid_l e grid_r
    # Precisa importar matplotlib.pyplot as plt no início do script
    import matplotlib.pyplot as plt
    
    if data.size == 0:
        print(f"Dados da fatia {slice_idx_display} estão vazios. Não plotando.")
        return

    plt.figure(figsize=(10,5))
    plt.imshow(data, cmap='gray')
    plt.title(f"Slice {slice_idx_display}")

    for rect_list, color in [(grid_l, 'red'), (grid_r, 'blue')]:
        if not rect_list: # Se a lista de retângulos está vazia
            continue
        for rect_coords in rect_list:
            y1, y2, x1, x2 = rect_coords # Coordenadas inclusivas
            largura_rect = x2 - x1 + 1
            altura_rect = y2 - y1 + 1
            # plt.Rectangle usa (x_inferior_esquerdo, y_inferior_esquerdo)
            # O y do matplotlib é geralmente de cima para baixo como no array numpy.
            plt.gca().add_patch(
                plt.Rectangle((x1, y1), largura_rect, altura_rect, 
                              linewidth=1, edgecolor=color, facecolor='none')
            )
    plt.show()

## Main Loop que gera os recortes de cada fatia

##### Só o T1

In [5]:
# informações pra analisar dados
imagens = "Patients_Displasya/T1"
mascara = "Mascaras"
coordinates_path = f"Coordenadas_grid"

total_label1 = []
excluded_patients = ["sub-54K08", "sub-87G01", "sub-89A03", "sub-90K10"]

for img, mask in zip([f for f in sorted(os.listdir(imagens)) if f.endswith(('.nii', '.nii.gz'))], [f for f in sorted(os.listdir(mascara)) if f.endswith(('.nrrd', '.nii', '.nii.gz'))]):
    if (img.split('_')[0] in excluded_patients or mask.split(' ')[0] in excluded_patients):
        continue
    
    data = nib.load(os.path.join(imagens, img)).get_fdata()
    lesion_data, _ = nrrd.read(os.path.join(mascara, mask))
    
    # Rotacionar as imagens e as máscaras 90 graus
    data = np.rot90(data, k=1)
    lesion_data = np.rot90(lesion_data, k=1)
    
    # Verifica o formato das imagens
    print(data[2].shape)
    print(lesion_data[2].shape)
    
    # Contador para acompanhar quantas fatias foram processadas
    processed_slices = 0

    # Diretório de saída para salvar as fatias
    output_dir_left = os.path.join(f"Novo_Contralateral/Contralateral_T1/{img.split('_')[0]}", "left")
    output_dir_right = os.path.join(f"Novo_Contralateral/Contralateral_T1/{img.split('_')[0]}", "right")
    output_dir_lesion_left = os.path.join(f"Novo_Contralateral/Contralateral_T1/{mask.split(' ')[0]}", "lesion_left")
    output_dir_lesion_right = os.path.join(f"Novo_Contralateral/Contralateral_T1/{mask.split(' ')[0]}", "lesion_right")
    os.makedirs(output_dir_left, exist_ok=True)
    os.makedirs(output_dir_right, exist_ok=True)
    os.makedirs(output_dir_lesion_left, exist_ok=True)
    os.makedirs(output_dir_lesion_right, exist_ok=True)

    coordinates_path = f"Coordenadas_grid/{img.split('_')[0]}"
    os.makedirs(coordinates_path, exist_ok=True)
    
    # Loop para cada fatia axial
    for slice_idx in range(data.shape[2]):
        if slice_idx>15 and slice_idx<150:
            lesion_slice_data = lesion_data[:, :, slice_idx]
            lesion_slice_data = np.where(lesion_slice_data>0.6, 1, 0)
            
            #slice_data = data[16:233-17, 18:197-19, slice_idx]
            slice_data = data[:, :, slice_idx]
            
            total_pixels = slice_data.size
            non_zero_pixels = np.count_nonzero(slice_data)
            non_black_ratio = non_zero_pixels / total_pixels if total_pixels > 0 else 0
            
            if non_black_ratio >= 0.05:
                left_grid, right_grid = create_full_grid_no_mirror(slice_data, 40, 35, 0.05, 0.1)

                left_subimages = []
                right_subimages = []

                # Para o lado esquerdo
                for idx, rect in enumerate(left_grid):
                    y1, y2, x1, x2 = rect
                    subimage = slice_data[y1:y2+1, x1:x2+1]
                    position = f"left_{idx + 1:03}"
                    left_subimages.append((subimage, position))

                # Para o lado direito
                for idx, rect in enumerate(right_grid):
                    y1, y2, x1, x2 = rect
                    subimage = slice_data[y1:y2+1, x1:x2+1]
                    position = f"right_{idx + 1:03}"
                    right_subimages.append((subimage, position))

                if len(left_subimages) == 0 or len(right_subimages) == 0 :
                    print(f"{img.split('_')[0]} - left:{len(left_subimages)} | right:{len(right_subimages)}")
                    continue
                
                output_dir_left_slice = os.path.join(output_dir_left, f"Slice_{slice_idx:03}/")
                output_dir_right_slice = os.path.join(output_dir_right, f"Slice_{slice_idx:03}/")

                os.makedirs(output_dir_left_slice, exist_ok=True)
                os.makedirs(output_dir_right_slice, exist_ok=True)
                
                processed_slices += 1
                
                # Salvar subimagens para cada lado
                for subimage, position in left_subimages:
                    output_path = os.path.join(output_dir_left_slice, f"{position}.nii.gz")
                    subimage_nii = nib.Nifti1Image(subimage, affine=np.eye(4))
                    nib.save(subimage_nii, output_path)

                for subimage, position in right_subimages:
                    output_path = os.path.join(output_dir_right_slice, f"{position}.nii.gz")
                    subimage_nii = nib.Nifti1Image(subimage, affine=np.eye(4))
                    nib.save(subimage_nii, output_path)

                # Aplicar o mesmo grid para lesion_slice_data
                left_subimages_lesion = []
                right_subimages_lesion = []
                
                # Para o lado esquerdo
                for idx, rect in enumerate(left_grid):
                    y1, y2, x1, x2 = rect
                    subimage_lesion = lesion_slice_data[y1:y2+1, x1:x2+1]
                    position = f"left_{idx + 1:03}"
                    left_subimages_lesion.append((subimage_lesion, position))

                # Para o lado direito
                for idx, rect in enumerate(right_grid):
                    y1, y2, x1, x2 = rect
                    subimage_lesion = lesion_slice_data[y1:y2+1, x1:x2+1]
                    position = f"right_{idx + 1:03}"
                    right_subimages_lesion.append((subimage_lesion, position))
                    
                output_dir_lesion_left_slice = os.path.join(output_dir_lesion_left, f"Slice_{slice_idx:03}")
                output_dir_lesion_right_slice = os.path.join(output_dir_lesion_right, f"Slice_{slice_idx:03}")
                
                os.makedirs(output_dir_lesion_left_slice, exist_ok=True)
                os.makedirs(output_dir_lesion_right_slice, exist_ok=True)
                
                # Salvar subimagens para cada lado
                for subimage, position in left_subimages_lesion:
                    output_path = os.path.join(output_dir_lesion_left_slice, f"{position}.nii.gz")
                    subimage_nii = nib.Nifti1Image(subimage, affine=np.eye(4), dtype=np.int64)
                    nib.save(subimage_nii, output_path)

                for subimage, position in right_subimages_lesion:
                    output_path = os.path.join(output_dir_lesion_right_slice, f"{position}.nii.gz")
                    subimage_nii = nib.Nifti1Image(subimage, affine=np.eye(4), dtype=np.int64)
                    nib.save(subimage_nii, output_path)
                
                coordinates_path = f"Coordenadas_grid/{img.split('_')[0]}/Slice_{slice_idx:03}.txt"
                with open(coordinates_path, 'w') as f:
                    for rect in (left_grid+right_grid):
                        y1, y2, x1, x2 = rect
                        f.write(f"{y1},{y2},{x1},{x2}\n")

    print(f"Paciente {img.split('_')[0]} recortado com sucesso!")

(197, 189)
(197, 189)
Paciente sub-00H10 recortado com sucesso!
(197, 189)
(197, 189)
Paciente sub-02A13 recortado com sucesso!
(197, 189)
(197, 189)
Paciente sub-06C09 recortado com sucesso!
(197, 189)
(197, 189)
Paciente sub-14F04 recortado com sucesso!
(197, 189)
(197, 189)


KeyboardInterrupt: 

##### Todas as 3 modalidades: T1, Flair, T2

In [8]:
# Informações pra analisar dados dos pacientes
imagens_base_paths = ["Patients_Displasya/T1", "Patients_Displasya/Flair", "Patients_Displasya/T2"]
modalities_output_names = ["Contralateral_T1", "Contralateral_Flair", "Contralateral_T2"]
mascara_base_path = "Novo_Mascaras"

for mod_name in modalities_output_names:
    os.makedirs(f"Novo_Contralateral/{mod_name}", exist_ok=True)

coordinates_dir_base = "Novas_Coordenadas_grid"
os.makedirs(coordinates_dir_base, exist_ok=True)

pacient_t1_files = sorted([f for f in os.listdir(imagens_base_paths[0]) if f.endswith(('.nii', '.nii.gz'))])
pacient_flair_files = sorted([f for f in os.listdir(imagens_base_paths[1]) if f.endswith(('.nii', '.nii.gz'))])
pacient_t2_files = sorted([f for f in os.listdir(imagens_base_paths[2]) if f.endswith(('.nii', '.nii.gz'))])

if not (len(pacient_t1_files) == len(pacient_flair_files) == len(pacient_t2_files)):
    print("AVISO: O número de arquivos T1, Flair e T2 não é o mesmo. Verifique os diretórios de entrada.")

for index in range(len(pacient_t1_files)):
    t1_file_name = pacient_t1_files[index]
    if index >= len(pacient_flair_files) or index >= len(pacient_t2_files):
        print(f"Pulando {t1_file_name} devido à falta de arquivos correspondentes Flair/T2 (índice {index} fora dos limites).")
        continue
    flair_file_name = pacient_flair_files[index]
    t2_file_name = pacient_t2_files[index]
    
    patient_id = t1_file_name.split('_')[0]
    print(f"\nProcessando Paciente: {patient_id}")

    try:
        mask_file_name = None
        for f_mask in os.listdir(mascara_base_path):
            if f_mask.startswith(patient_id) and f_mask.endswith(('.nrrd', '.nii', '.nii.gz')):
                mask_file_name = f_mask
                break
        
        if not mask_file_name:
            print(f"  Máscara não encontrada para o paciente {patient_id}. Pulando paciente.")
            continue

        data_t1_full = nib.load(os.path.join(imagens_base_paths[0], t1_file_name)).get_fdata()
        data_flair_full = nib.load(os.path.join(imagens_base_paths[1], flair_file_name)).get_fdata()
        data_t2_full = nib.load(os.path.join(imagens_base_paths[2], t2_file_name)).get_fdata()
        lesion_data_full, _ = nrrd.read(os.path.join(mascara_base_path, mask_file_name))

        all_modalities_data = {
            modalities_output_names[0]: data_t1_full,
            modalities_output_names[1]: data_flair_full,
            modalities_output_names[2]: data_t2_full,
        }

        for key in all_modalities_data:
            all_modalities_data[key] = np.rot90(all_modalities_data[key], k=1)
        lesion_data_full = np.rot90(lesion_data_full, k=1)

        num_slices_t1 = data_t1_full.shape[2]
        if not (data_flair_full.shape[2] == num_slices_t1 and \
                data_t2_full.shape[2] == num_slices_t1 and \
                lesion_data_full.shape[2] == num_slices_t1):
            print(f"  AVISO: Inconsistência no número de fatias para o paciente {patient_id}. Pulando.")
            print(f"    Slices T1: {num_slices_t1}, Flair: {data_flair_full.shape[2]}, T2: {data_t2_full.shape[2]}, Lesão: {lesion_data_full.shape[2]}")
            continue

        patient_coordinates_path_output = os.path.join(coordinates_dir_base, patient_id)
        os.makedirs(patient_coordinates_path_output, exist_ok=True)

        num_slices = num_slices_t1

        for slice_idx in range(num_slices):
            slice_t1_orig = all_modalities_data[modalities_output_names[0]][:, :, slice_idx]
            slice_flair_orig = all_modalities_data[modalities_output_names[1]][:, :, slice_idx]
            slice_t2_orig = all_modalities_data[modalities_output_names[2]][:, :, slice_idx]

            combined_slice_for_grid_logic_binary = np.zeros_like(slice_t1_orig, dtype=np.uint8)
            combined_slice_for_grid_logic_binary[slice_t1_orig > 0] = 1
            combined_slice_for_grid_logic_binary[slice_flair_orig > 0] = 1
            combined_slice_for_grid_logic_binary[slice_t2_orig > 0] = 1
            
            slice_data_ref_for_grid_decision = combined_slice_for_grid_logic_binary

            total_pixels_ref = slice_data_ref_for_grid_decision.size
            non_zero_pixels_ref = np.count_nonzero(slice_data_ref_for_grid_decision)
            non_black_ratio_ref = non_zero_pixels_ref / total_pixels_ref if total_pixels_ref > 0 else 0

            left_grid_comum, right_grid_comum = [], []

            if non_black_ratio_ref >= 0.05:
                left_grid_comum, right_grid_comum = create_full_grid_no_mirror(
                    slice_data_ref_for_grid_decision, 40, 35, 0.05, 0.1
                )

                if not left_grid_comum and not right_grid_comum:
                    continue 
                
                slice_coordinates_file = os.path.join(patient_coordinates_path_output, f"Slice_{slice_idx:03}.txt")
                with open(slice_coordinates_file, 'w') as f_coord:
                    f_coord.write("# Left Grid\n")
                    for rect_coords in left_grid_comum: # Renomeado para clareza
                        f_coord.write(f"{rect_coords[0]},{rect_coords[1]},{rect_coords[2]},{rect_coords[3]}\n")
                    f_coord.write("# Right Grid\n")
                    for rect_coords in right_grid_comum: # Renomeado para clareza
                        f_coord.write(f"{rect_coords[0]},{rect_coords[1]},{rect_coords[2]},{rect_coords[3]}\n")
            else:
                continue

            lesion_slice_data_processed = np.where(lesion_data_full[:, :, slice_idx] > 0.7, 1, 0).astype(np.int8)

            for mod_output_name in modalities_output_names:
                current_slice_data_orig = all_modalities_data[mod_output_name][:, :, slice_idx]

                output_dir_img_left_base = os.path.join(f"Novo_Contralateral/{mod_output_name}/{patient_id}", "left")
                output_dir_img_right_base = os.path.join(f"Novo_Contralateral/{mod_output_name}/{patient_id}", "right")
                os.makedirs(output_dir_img_left_base, exist_ok=True)
                os.makedirs(output_dir_img_right_base, exist_ok=True)
                output_dir_img_left_slice = os.path.join(output_dir_img_left_base, f"Slice_{slice_idx:03}")
                output_dir_img_right_slice = os.path.join(output_dir_img_right_base, f"Slice_{slice_idx:03}")
                os.makedirs(output_dir_img_left_slice, exist_ok=True)
                os.makedirs(output_dir_img_right_slice, exist_ok=True)

                output_dir_lesion_left_base = os.path.join(f"Novo_Contralateral/{mod_output_name}/{patient_id}", "lesion_left")
                output_dir_lesion_right_base = os.path.join(f"Novo_Contralateral/{mod_output_name}/{patient_id}", "lesion_right")
                os.makedirs(output_dir_lesion_left_base, exist_ok=True)
                os.makedirs(output_dir_lesion_right_base, exist_ok=True)
                output_dir_lesion_left_slice = os.path.join(output_dir_lesion_left_base, f"Slice_{slice_idx:03}")
                output_dir_lesion_right_slice = os.path.join(output_dir_lesion_right_base, f"Slice_{slice_idx:03}")
                os.makedirs(output_dir_lesion_left_slice, exist_ok=True)
                os.makedirs(output_dir_lesion_right_slice, exist_ok=True)

                for side_label, grid_para_lado in [("left", left_grid_comum), ("right", right_grid_comum)]:
                    target_output_img_slice_dir = output_dir_img_left_slice if side_label == "left" else output_dir_img_right_slice
                    target_output_lesion_slice_dir = output_dir_lesion_left_slice if side_label == "left" else output_dir_lesion_right_slice
                    
                    if not grid_para_lado: # Checa se o grid específico (left ou right) está vazio
                        continue

                    for patch_idx, rect_coords in enumerate(grid_para_lado): # Renomeado para clareza
                        y1, y2, x1, x2 = rect_coords 
                        position = f"{side_label}_{patch_idx + 1:03}"

                        # Slicing para subimagem: y2 e x2 são inclusivos nas coordenadas, então +1 para slicing
                        subimage = current_slice_data_orig[y1 : y2 + 1, x1 : x2 + 1]
                        if subimage.size == 0: continue # Pula se o patch resultante for vazio
                        output_path_img = os.path.join(target_output_img_slice_dir, f"{position}.nii.gz")
                        subimage_nii_img = nib.Nifti1Image(subimage.astype(np.float32), affine=np.eye(4))
                        nib.save(subimage_nii_img, output_path_img)
                        
                        subimage_lesion = lesion_slice_data_processed[y1 : y2 + 1, x1 : x2 + 1]
                        if subimage_lesion.size == 0: continue # Pula se o patch de lesão for vazio
                        output_path_lesion = os.path.join(target_output_lesion_slice_dir, f"{position}.nii.gz")
                        subimage_nii_lesion = nib.Nifti1Image(subimage_lesion.astype(np.int8), affine=np.eye(4))
                        nib.save(subimage_nii_lesion, output_path_lesion)
            
        print(f"  Paciente {patient_id} processado com sucesso!")

    except FileNotFoundError as e:
        print(f"  Erro: Arquivo não encontrado para paciente {patient_id} (detalhes: {e}). Pulando paciente.")
    except Exception as e:
        print(f"  Erro inesperado ao processar paciente {patient_id} (detalhes: {e}).")
        import traceback
        traceback.print_exc()

print("\nProcessamento de todos os pacientes concluído.")


Processando Paciente: sub-00H10
  Paciente sub-00H10 processado com sucesso!

Processando Paciente: sub-02A13
  Paciente sub-02A13 processado com sucesso!

Processando Paciente: sub-06C09
  Paciente sub-06C09 processado com sucesso!

Processando Paciente: sub-14F04
  Paciente sub-14F04 processado com sucesso!

Processando Paciente: sub-16E03
  Paciente sub-16E03 processado com sucesso!

Processando Paciente: sub-16G09
  Máscara não encontrada para o paciente sub-16G09. Pulando paciente.

Processando Paciente: sub-16I12
  Paciente sub-16I12 processado com sucesso!

Processando Paciente: sub-19F09
  Paciente sub-19F09 processado com sucesso!

Processando Paciente: sub-19G04
  Paciente sub-19G04 processado com sucesso!

Processando Paciente: sub-26B09
  Paciente sub-26B09 processado com sucesso!

Processando Paciente: sub-29D03
  Paciente sub-29D03 processado com sucesso!

Processando Paciente: sub-31F07
  Paciente sub-31F07 processado com sucesso!

Processando Paciente: sub-35E12
  Paci

## Plots

In [6]:
patient1 = 'sub-72K02_ses-01_acq-mpr_run-01_desc-reg_desc-biascorr_desc-brainext_T1w.nii.gz'
patient2 = 'sub-86G08_ses-01_acq-posgado_desc-reg_desc-biascorr_desc-brainext_T1w.nii.gz'

patients = os.listdir('Patients_Displasya/T1')
patient = patients[9]
print(patient)

path = os.path.join('Patients_Displasya', patient)
data_3d = nib.load(path).get_fdata()
data_3d = np.rot90(data_3d, k=3)  # Rotaciona a imagem 90 graus

#print(patients)

for z in range(0, data_3d.shape[1]-1):    
    z = z+50
    if z > data_3d.shape[1]-1:
        break
    data_2d = data_3d[:, z, :]

    #print(f"{data_2d.shape} centro: {data_2d.shape[0]//2}, {data_2d.shape[1]//2}\n")

    grid_l, grid_r = create_full_grid_no_mirror(data_2d, 40, 35, 0.05, 0.1)
    
    #grid_l, grid_r = create_left_right_grid(data_2d, 40, 35, 0.05)
    #plot_grid(data_2d, grid_l+grid_r, z)
    #grid_l, grid_r = move_grid_full(data_2d, grid_l, grid_r, 0.1)
    full_grid = grid_l+grid_r

    #for idx in range(len(full_grid)//2):
    #    print(full_grid[idx], full_grid[idx+len(full_grid)//2])

    plot_grid(data_2d, full_grid, z)

sub-26B09_ses-01_acq-vol_desc-reg_desc-biascorr_desc-brainext_T1w.nii.gz


FileNotFoundError: No such file or no access: 'Patients_Displasya/sub-26B09_ses-01_acq-vol_desc-reg_desc-biascorr_desc-brainext_T1w.nii.gz'

In [ ]:
patch_l = nib.load(r"Novo_Contralateral\sub-00H10\left\Slice009\left_001.nii.gz")
patch_r = nib.load(r"Novo_Contralateral\sub-00H10\right\Slice009\right_001.nii.gz")

print(patch_l.shape, patch_r.shape)

In [ ]:
# informações pra analisar dados

img_path = 'Patients_Displasya/sub-06C09_ses-01_acq-vbm_desc-reg_desc-biascorr_desc-brainext_T1w.nii.gz'
z_idx = 95
img = nib.load(img_path)
data_3d = img.get_fdata()
data_3d = np.rot90(data_3d, k=3)  # Rotaciona a imagem 90 graus


print(data_3d.shape)
print(data_3d.shape[0])
print(data_3d.shape[1])
print(data_3d.shape[2])

plt.imshow(data_3d[:, :, z_idx], cmap='gray')
plt.show()

In [ ]:
#img_test = data_3d[16:233-17, 18:197-19, z_idx]  # Para deixar com dimensões (160, 200)
img_test = data_3d[:, :, z_idx+45]  # dimensões originais
print (img_test.shape)

left_grid, right_grid = create_left_right_grid(data_3d, 45, 35, 0.1)

# Separar subimagens em lados esquerdo e direito
left_subimages = []
right_subimages = []

# Para o lado esquerdo
for idx, rect in enumerate(left_grid):
    y1, y2, x1, x2 = rect
    subimage = data_3d[y1:y2+1, x1:x2+1]
    position = f"left_{idx + 1:03}"
    left_subimages.append((subimage, position))

# Para o lado direito
for idx, rect in enumerate(right_grid):
    y1, y2, x1, x2 = rect
    subimage = data_3d[y1:y2+1, x1:x2+1]
    position = f"right_{idx + 1:03}"
    right_subimages.append((subimage, position))

# Plotando a imagem com o grid sobreposto
fig, ax = plt.subplots(figsize=(10, 8))
ax.imshow(img_test, cmap='gray')

# Adicionando o grid vermelho ao lado esquerdo
for rect in left_grid:
    y1, y2, x1, x2 = rect
    rect_patch = plt.Rectangle((x1, y1), x2 - x1, y2 - y1, edgecolor='red', facecolor='none', linewidth=1)
    ax.add_patch(rect_patch)

ax.axis("off")

plt.show()

In [ ]:
for item in left_grid:
    print(item)
for item in right_grid:
    print(item)
# 0 alinhado ao 6
# i -> n//2 = 0 -> 

In [ ]:
plt.imshow(img_test, cmap='gray')

adjusted_grid_l, adjusted_grid_r = move_grid_full(img_test, left_grid, right_grid) # move cada quadradinho para encaixar no cérebro, minimizando espaços pretos

plot_grid(img_test, adjusted_grid_l, z_idx)
plot_grid(img_test, adjusted_grid_r, z_idx)
plot_grid(img_test, adjusted_grid_l+adjusted_grid_r, z_idx)

In [ ]:
for z in range(0, data_3d.shape[2]-1):
        data_2d = data_3d[:, :, z]
        
        moved, mirror = create_full_grid(data_2d, 40, 35, 0.1)

        full_grid = moved+mirror

        if len(full_grid) != 0:

            plt.imshow(data_2d, cmap='gray')
            plt.title(f"z={z} n_grid={len(full_grid)}")

            for rect in (full_grid):
            
                y1, y2, x1, x2 = rect[0], rect[1], rect[2], rect[3]

                plt.gca().add_patch(plt.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=1, edgecolor='red', facecolor='none'))

            plt.show()